# pre_Data-Transform
### Step_1. Formatting2Yolo

In [4]:
!python Code/pre_Formatting2yolo.py

Formatting Official Labels to yolo Type...
100%|███████████████████████████████████████| 1000/1000 [00:39<00:00, 25.39it/s]


In [ ]:
# from code.pre_Formatting2yolo import pre_Formatting2yolo
#
# pre_Formatting2yolo()
